In [4]:
import os
import sys
import numpy as np
import pandas as pd
import torch
import gpytorch
import matplotlib.pyplot as plt
import tqdm
import utils as utils
import scienceplots
plt.style.use('science')

base_dir = os.path.join(os.getcwd(), '..')
sys.path.append(base_dir)

import fit_FaIRGP as fairgp
import fit_Plain_GP as plaingp
import evaluate_FaIRGP as evalFaIRGP
import evaluate_Plain_GP as evalGP
import evaluate_FaIR as evalFaIR

In [5]:
train_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['historical']}}
train_data = plaingp.make_data(train_cfg)

test_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['ssp126', 'ssp245', 'ssp370', 'ssp585']}}
test_data = plaingp.make_data(test_cfg)

gpmodel = plaingp.make_model(train_cfg, train_data)
state_dict = torch.load('../data/models/historical-only/PlainGP/state_dict.pt')
gpmodel.load_state_dict(state_dict)

fairgpmodel = fairgp.make_model(train_cfg, train_data)
state_dict = torch.load('../data/models/historical-only/FaIRGP/state_dict.pt')
fairgpmodel.load_state_dict(state_dict)

<All keys matched successfully>

In [6]:
test_scenarios = test_data.scenarios
test_times = test_scenarios.timesteps
test_tas = test_scenarios.tas
test_tas_fair = fairgpmodel._compute_mean(test_scenarios)
X_test = torch.cat([test_scenarios.cum_emissions[:, 0, None],
                    test_scenarios.emissions[:, 1:]], dim=-1)
X_test = (X_test - gpmodel.mu) / gpmodel.sigma

gpmodel = gpmodel.eval()
fairgpmodel = fairgpmodel.eval()

with torch.no_grad():
    noisy_gp_test_posterior = gpmodel.likelihood(gpmodel(X_test))
    mean = gpmodel.sigma_targets * noisy_gp_test_posterior.mean
    covar = gpmodel.sigma_targets.pow(2) * noisy_gp_test_posterior.covariance_matrix
    noisy_gp_test_posterior = noisy_gp_test_posterior.__class__(mean, covar)
    
with torch.no_grad():
    fairgp_test_posterior = fairgpmodel(test_scenarios)
    noisy_fairgp_test_posterior = fairgpmodel.likelihood(fairgp_test_posterior)
    mean = noisy_fairgp_test_posterior.mean + test_tas_fair
    covar = noisy_fairgp_test_posterior.lazy_covariance_matrix
    noisy_fairgp_test_posterior = noisy_fairgp_test_posterior.__class__(mean, covar)

fair_tas = []
for scenario in test_scenarios.scenarios.values():
    output = evalFaIR.run_fair(scenario=scenario, cfg=test_cfg)
    fair_tas.append(output)
fair_tas = np.concatenate(fair_tas)

/home/bouabid/.pyenv/versions/sandbox/lib/python3.8/site-packages/gpytorch/lazy/lazy_tensor.py:25: UserWarning: gpytorch.lazy.delazify is deprecated in favor of linear_operator.to_dense
  warnings.warn("gpytorch.lazy.delazify is deprecated in favor of linear_operator.to_dense")
/home/bouabid/.pyenv/versions/sandbox/lib/python3.8/site-packages/gpytorch/lazy/non_lazy_tensor.py:13: UserWarning: gpytorch.lazy.lazify is deprecated in favor of linear_operator.to_linear_operator
  warnings.warn("gpytorch.lazy.lazify is deprecated in favor of linear_operator.to_linear_operator")


In [7]:
fair_scores = evalFaIR.evaluate(fair_tas=fair_tas, groundtruth_tas=test_scenarios.tas)
gp_scores = evalGP.compute_scores(noisy_gp_test_posterior, test_scenarios)
fairgp_scores = evalFaIRGP.compute_scores(noisy_fairgp_test_posterior, test_scenarios)

In [8]:
scores_df = pd.DataFrame([fair_scores, gp_scores, fairgp_scores])
scores_df.columns = ['Bias', 'RMSE', 'MAE', 'Corr', 'LL', 'Calib95', 'CRPS', 'ICI']
scores_df.index = ['FaIR', 'Plain GP', 'FaIRGP']
scores_df

,Bias,RMSE,MAE,Corr,LL,Calib95,CRPS,ICI
FaIR,0.072403,0.220294,0.177461,0.975784,NaN,NaN,NaN,NaN
Plain GP,-1.587697,1.800609,1.587697,-0.220317,-39.498875,0.026163,1.474090,0.496787
FaIRGP,-0.046000,0.188821,0.141061,0.978468,0.304038,0.918605,0.101539,0.013158


In [11]:
results = scores_df[['RMSE', 'MAE', 'Bias', 'LL', 'Calib95', 'CRPS']].round(3).fillna('-')
# results.to_latex('./tables/historical-only-scores.tex')
results

/tmp/ipykernel_10514/713675530.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  results.to_latex('./tables/historical-only-scores.tex')


,RMSE,MAE,Bias,LL,Calib95,CRPS
FaIR,0.220,0.177,0.072,-,-,-
Plain GP,1.801,1.588,-1.588,-39.499,0.026,1.474
FaIRGP,0.189,0.141,-0.046,0.304,0.919,0.102
